In [1]:
"""
🎮 GOON SIMULATOR 2079 🎮
A cyberpunk arcade game where you're just trying to make it as a goon in Neo-City
"""
import pygame
import random
import math

# Initialize Pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("🔥 GOON SIMULATOR 2079 🔥")

# Colors (Cyberpunk palette)
NEON_PINK = (255, 20, 147)
NEON_BLUE = (0, 255, 255)
NEON_PURPLE = (148, 0, 211)
NEON_GREEN = (57, 255, 20)
DARK_BG = (15, 15, 35)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# Fonts
font_large = pygame.font.Font(None, 74)
font_medium = pygame.font.Font(None, 48)
font_small = pygame.font.Font(None, 36)

class Goon:
    def __init__(self):
        self.x = WIDTH // 2
        self.y = HEIGHT // 2
        self.size = 30
        self.speed = 5
        self.respect = 0
        self.energy = 100
        self.swag_level = 1
        self.color = NEON_PINK
        
    def move(self, keys):
        if keys[pygame.K_LEFT] or keys[pygame.K_a]:
            self.x -= self.speed
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
            self.x += self.speed
        if keys[pygame.K_UP] or keys[pygame.K_w]:
            self.y -= self.speed
        if keys[pygame.K_DOWN] or keys[pygame.K_s]:
            self.y += self.speed
        # Keep in bounds
        self.x = max(self.size, min(WIDTH - self.size, self.x))
        self.y = max(self.size, min(HEIGHT - self.size, self.y))
        
    def draw(self, surface):
        # Draw glow effect
        for i in range(3, 0, -1):
            alpha = 50 * i
            glow_surf = pygame.Surface((self.size*4, self.size*4), pygame.SRCALPHA)
            pygame.draw.circle(glow_surf, (*self.color[:3], alpha), 
                             (self.size*2, self.size*2), self.size + i*5)
            surface.blit(glow_surf, (self.x - self.size*2, self.y - self.size*2))
        # Draw goon (cool shades wearing circle)
        pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), self.size)
        pygame.draw.circle(surface, WHITE, (int(self.x), int(self.y)), self.size, 2)
        # Sunglasses (very important for a goon)
        pygame.draw.rect(surface, (0, 0, 0), (self.x - 15, self.y - 8, 12, 6))
        pygame.draw.rect(surface, (0, 0, 0), (self.x + 3, self.y - 8, 12, 6))
        pygame.draw.line(surface, (0, 0, 0), (self.x - 3, self.y - 5), (self.x + 3, self.y - 5), 2)

class Collectible:
    def __init__(self, item_type):
        self.x = random.randint(50, WIDTH - 50)
        self.y = random.randint(80, HEIGHT - 50)
        self.type = item_type
        self.size = 20
        self.pulse = 0
        
        if item_type == "cash":
            self.color = NEON_GREEN
            self.value = random.randint(10, 50)
            self.symbol = "$"
        elif item_type == "energy_drink":
            self.color = NEON_BLUE
            self.value = 25
            self.symbol = "⚡"
        elif item_type == "swag":
            self.color = NEON_PURPLE
            self.value = 1
            self.symbol = "★"
            
    def draw(self, surface):
        self.pulse = (self.pulse + 0.1) % (2 * math.pi)
        size_mod = math.sin(self.pulse) * 3
        
        # Glow
        glow_surf = pygame.Surface((80, 80), pygame.SRCALPHA)
        pygame.draw.circle(glow_surf, (*self.color[:3], 50), (40, 40), int(self.size + size_mod + 10))
        surface.blit(glow_surf, (self.x - 40, self.y - 40))
        
        pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), int(self.size + size_mod))
        text = font_small.render(self.symbol, True, WHITE)
        surface.blit(text, (self.x - text.get_width()//2, self.y - text.get_height()//2))

class Obstacle:
    def __init__(self):
        self.x = random.choice([0, WIDTH])
        self.y = random.randint(80, HEIGHT - 50)
        self.speed = random.uniform(2, 4)
        self.direction = 1 if self.x == 0 else -1
        self.size = 25
        
    def move(self):
        self.x += self.speed * self.direction
        
    def draw(self, surface):
        # Cop drone
        pygame.draw.polygon(surface, RED, [
            (self.x, self.y - self.size),
            (self.x - self.size, self.y + self.size//2),
            (self.x + self.size, self.y + self.size//2)
        ])
        pygame.draw.circle(surface, YELLOW, (int(self.x), int(self.y)), 8)
        # Flashing light
        if pygame.time.get_ticks() % 500 < 250:
            pygame.draw.circle(surface, RED, (int(self.x), int(self.y - self.size + 5)), 5)

def draw_background(surface, scroll):
    surface.fill(DARK_BG)
    # Grid effect
    for x in range(0, WIDTH, 40):
        alpha = 30 + int(20 * math.sin((x + scroll) * 0.05))
        pygame.draw.line(surface, (*NEON_PURPLE[:3],), (x, 0), (x, HEIGHT), 1)
    for y in range(0, HEIGHT, 40):
        pygame.draw.line(surface, (*NEON_BLUE[:3],), (0, y), (WIDTH, y), 1)

def draw_hud(surface, goon, high_score):
    # Top bar
    pygame.draw.rect(surface, (30, 30, 60), (0, 0, WIDTH, 70))
    pygame.draw.line(surface, NEON_PINK, (0, 70), (WIDTH, 70), 2)
    
    # Stats
    respect_text = font_small.render(f"RESPECT: ${goon.respect}", True, NEON_GREEN)
    energy_text = font_small.render(f"ENERGY: {int(goon.energy)}%", True, NEON_BLUE)
    swag_text = font_small.render(f"SWAG LVL: {goon.swag_level}", True, NEON_PURPLE)
    high_text = font_small.render(f"HIGH: ${high_score}", True, YELLOW)
    
    surface.blit(respect_text, (20, 20))
    surface.blit(energy_text, (220, 20))
    surface.blit(swag_text, (450, 20))
    surface.blit(high_text, (650, 20))

def game_over_screen(surface, score):
    overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 180))
    surface.blit(overlay, (0, 0))
    
    title = font_large.render("BUSTED!", True, RED)
    subtitle = font_medium.render(f"Final Respect: ${score}", True, WHITE)
    restart = font_small.render("Press SPACE to try again", True, NEON_BLUE)
    
    surface.blit(title, (WIDTH//2 - title.get_width()//2, HEIGHT//2 - 80))
    surface.blit(subtitle, (WIDTH//2 - subtitle.get_width()//2, HEIGHT//2))
    surface.blit(restart, (WIDTH//2 - restart.get_width()//2, HEIGHT//2 + 60))

def title_screen(surface):
    surface.fill(DARK_BG)
    # Animated background
    for i in range(20):
        x = random.randint(0, WIDTH)
        y = random.randint(0, HEIGHT)
        pygame.draw.circle(surface, (*NEON_PURPLE[:3], 100), (x, y), random.randint(1, 3))
    
    title = font_large.render("GOON SIMULATOR", True, NEON_PINK)
    year = font_large.render("2079", True, NEON_BLUE)
    subtitle = font_small.render("Collect cash 💰 | Grab energy drinks ⚡ | Avoid the cops!", True, WHITE)
    start = font_medium.render("Press SPACE to start your grind", True, NEON_GREEN)
    controls = font_small.render("WASD or Arrow Keys to move", True, YELLOW)
    
    surface.blit(title, (WIDTH//2 - title.get_width()//2, 150))
    surface.blit(year, (WIDTH//2 - year.get_width()//2, 220))
    surface.blit(subtitle, (WIDTH//2 - subtitle.get_width()//2, 320))
    surface.blit(start, (WIDTH//2 - start.get_width()//2, 400))
    surface.blit(controls, (WIDTH//2 - controls.get_width()//2, 480))

def main():
    clock = pygame.time.Clock()
    running = True
    game_state = "title"  # title, playing, game_over
    high_score = 0
    scroll = 0
    
    goon = Goon()
    collectibles = []
    obstacles = []
    spawn_timer = 0
    obstacle_timer = 0
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if game_state == "title":
                        game_state = "playing"
                        goon = Goon()
                        collectibles = []
                        obstacles = []
                    elif game_state == "game_over":
                        game_state = "playing"
                        goon = Goon()
                        collectibles = []
                        obstacles = []
        
        if game_state == "title":
            title_screen(screen)
            
        elif game_state == "playing":
            scroll += 1
            keys = pygame.key.get_pressed()
            goon.move(keys)
            
            # Spawn collectibles
            spawn_timer += 1
            if spawn_timer > 60:
                spawn_timer = 0
                item_type = random.choices(["cash", "energy_drink", "swag"], weights=[60, 30, 10])[0]
                collectibles.append(Collectible(item_type))
                if len(collectibles) > 10:
                    collectibles.pop(0)
            
            # Spawn obstacles (more as you level up)
            obstacle_timer += 1
            if obstacle_timer > max(120 - goon.swag_level * 10, 40):
                obstacle_timer = 0
                obstacles.append(Obstacle())
                if len(obstacles) > 5 + goon.swag_level:
                    obstacles.pop(0)
            
            # Move obstacles
            for obs in obstacles[:]:
                obs.move()
                if obs.x < -50 or obs.x > WIDTH + 50:
                    obstacles.remove(obs)
            
            # Check collisions with collectibles
            for item in collectibles[:]:
                dist = math.sqrt((goon.x - item.x)**2 + (goon.y - item.y)**2)
                if dist < goon.size + item.size:
                    if item.type == "cash":
                        goon.respect += item.value * goon.swag_level
                    elif item.type == "energy_drink":
                        goon.energy = min(100, goon.energy + item.value)
                    elif item.type == "swag":
                        goon.swag_level += 1
                        goon.color = random.choice([NEON_PINK, NEON_BLUE, NEON_GREEN, NEON_PURPLE])
                    collectibles.remove(item)
            
            # Check collisions with obstacles
            for obs in obstacles:
                dist = math.sqrt((goon.x - obs.x)**2 + (goon.y - obs.y)**2)
                if dist < goon.size + obs.size:
                    goon.energy -= 30
            
            # Drain energy over time
            goon.energy -= 0.05
            
            # Check game over
            if goon.energy <= 0:
                game_state = "game_over"
                high_score = max(high_score, goon.respect)
            
            # Draw everything
            draw_background(screen, scroll)
            for item in collectibles:
                item.draw(screen)
            for obs in obstacles:
                obs.draw(screen)
            goon.draw(screen)
            draw_hud(screen, goon, high_score)
            
        elif game_state == "game_over":
            draw_background(screen, scroll)
            game_over_screen(screen, goon.respect)
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()

# Run the game!
main()

C:\Users\jackl\AppData\Roaming\Python\Python313\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
